<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab)V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2090, done.
remote: Counting objects: 100% (908/908), done.
remote: Compressing objects: 100% (450/450), done.
remote: Total 2090 (delta 596), reused 686 (delta 449), pack-reused 1182
Receiving objects: 100% (2090/2090), 48.58 MiB | 25.21 MiB/s, done.
Resolving deltas: 100% (1374/1374), done.


# import

In [2]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# demo data

In [3]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
140-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.0911,20.854397,74.689774,70,980,1.75,10,36.0


In [4]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
140-023,980


In [5]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
140-023,70


In [6]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
140-023,10


# 1.試算模式

In [7]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [8]:
demo['icg_input']

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
140-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.0911,20.854397,74.689774,70,980,1.75,10,36.0


In [9]:
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

(1, 3)
(1, 41)
(1, 10)
(1, 2)
(1, 41)
(1, 10)


# 檢查三項約束條件

In [10]:
c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023,69.163988


In [11]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

981.4707079837214

In [12]:
c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

9.062412707267518

# 推薦模式

In [13]:
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2,bz_error,nainbz_error,tol_error = f.recommend(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

[I 2021-04-23 07:00:58,628] A new study created in memory with name: no-name-d8f24dde-009d-41ca-a288-2126129cd4f8
[I 2021-04-23 07:00:58,672] Trial 0 finished with value: 9.4389145599079 and parameters: {'Density_Feed Properties': 0.8637547933543552, 'Density_Vent Gas Production Rate and Composition': 0.44145679406474125, 'Density_Distillate Production Rate and Composition': 0.7931169411614516, 'Density_Sidedraw Production Rate and Composition': 0.8805395121647108, 'Density_Bottoms Production Rate and Composition': 0.8718838730376585, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 103.87865242138056, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 11.70318397993223, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 13.718669053653963, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 175.42612322023962, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 191.

(1, 3)
(1, 41)
(1, 10)
(1, 2)
(1, 41)
(1, 10)


[I 2021-04-23 07:00:58,730] Trial 1 finished with value: 0.8250682241191214 and parameters: {'Density_Feed Properties': 0.8634475949909015, 'Density_Vent Gas Production Rate and Composition': 0.4431332130539541, 'Density_Distillate Production Rate and Composition': 0.7996038953581207, 'Density_Sidedraw Production Rate and Composition': 0.8801894183070363, 'Density_Bottoms Production Rate and Composition': 0.8720665105591595, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 124.18003084355028, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 12.88443053619011, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 14.683178412884702, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 176.32223020197222, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.16614793190666}. Best is trial 1 with value: 0.8250682241191214.
[I 2021-04-23 07:00:58,771] Trial 2 finished wi

(1, 3)
(1, 41)
(1, 10)
bz_error: 0.00024121466937288005


[I 2021-04-23 07:01:20,404] Trial 2 finished with value: 70.36534962954386 and parameters: {'Density_Feed Properties': 0.8762516836741001, 'Density_Vent Gas Production Rate and Composition': 0.5445857391929154, 'Density_Distillate (Benzene Drag) Production Rate and Composition': 0.8669910593842579, 'Density_Sidedraw (Benzene )Production Rate and Composition': 0.8836906246381001, 'Density_Bottoms Production Rate and Composition': 0.8707226783284728, 'Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr': 119.45763132635352, 'Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 11.683134352938753, 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 10.896323154530974, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 86.52804411551331, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 89.63433324219028, 'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': 979.9999909394409}. B

(1, 2)
(1, 41)
(1, 10)
nainbz_error: 14.245057665144714
tol_error: 0.22970061430743982


In [14]:
bz_error,nainbz_error,tol_error

(0.00024121466937288005, 14.245057665144714, 0.22970061430743982)

# 計算delta調幅

In [15]:
c620_op2-c620_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
140-023,-0.001211,-0.002301,0.005805,0.001301,-0.000074,-8.065671,-0.829109,-1.08959,-1.675365,0.077745


In [16]:
c660_op2-c660_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
140-023,0.000061,0.00003,-0.00003,0.000052,0.000023,-0.000016,0.000037,-0.000034,-0.000081,0.000089


In [17]:
c670_op2-c670_op

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
140-023,5.960464e-08,-0.000012,2.481613,0.225819,0.201303,-0.009232,0.109421


# 檢查三項約束條件

In [18]:
c620_wt2[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023,70.000241


In [19]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt2.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

965.7549423348553

In [20]:
c660_wt2['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

10.22970061430744